In [42]:
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
import datetime
from langchain.llms import OpenAI
from langchain.output_parsers import DatetimeOutputParser
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback

In [43]:
class HistoryQuiz():
    system_message_prompt = SystemMessagePromptTemplate.from_template('You are a history buff who knows dates of all historical events')
    chat = ChatOpenAI()
    
    def create_history_question(self,topic):
        '''
        This method should output a historical question about the topic that has a date as the correct answer.
        For example:
        
            "On what date did World War 2 end?"
            
        '''
        human_message_prompt = HumanMessagePromptTemplate.from_template('Create a historical question about {topic} \
                                                                        that has date as the correct answer')
        chat_prompt = ChatPromptTemplate.from_messages([HistoryQuiz.system_message_prompt, human_message_prompt])
        request = chat_prompt.format_prompt(topic=topic).to_messages()
        # chat = ChatOpenAI()

        question = None
        with get_openai_callback() as cb:
            question = HistoryQuiz.chat(request, temperature=0.0)
            print(f"---- History Question ------\n{cb}\n----------")
       
        return question.content
    
    def get_AI_answer(self,question):
        '''
        This method should get the answer to the historical question from the method above.
        Note: This answer must be in datetime format! Use DateTimeOutputParser to confirm!
        
        September 2, 1945 --> datetime.datetime(1945, 9, 2, 0, 0)
        '''
        print(f"Question={question}")
        # date_format = "%Y-%m-%dT%H:%M:%S.%fZ"

        date_output_parser = DatetimeOutputParser()
        print(date_output_parser.get_format_instructions())
        human_template = """
        Answer the following question and return only the date as answer: {question}

        {format_instructions}
        """
        human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
        chat_prompt = ChatPromptTemplate.from_messages([HistoryQuiz.system_message_prompt, human_message_prompt])
        request = chat_prompt.format_prompt(question=question, format_instructions=date_output_parser.get_format_instructions()).to_messages()
        # chat = ChatOpenAI()

        topic_date_str = None
        with get_openai_callback() as cb:
            topic_date_str = HistoryQuiz.chat(request, temperature=0.0)
            print(f"----------\n{cb}\n----------")
        
        topic_date = date_output_parser.parse(topic_date_str.content)
        return topic_date
    
    def get_user_answer(self,question):
        '''
        This method should grab a user answer and convert it to datetime. It should collect a Year, Month, and Day.
        You can just use input() for this.
        '''
        print(f"Question={question}")
        
        user_year = int(input("Enter the year: "))
        user_month = int(input("Enter the month (1-12): "))
        user_day = int(input("Enter the day (1-31): "))

        return datetime.datetime(user_year, user_month, user_day)

    def check_user_answer(self,user_answer,ai_answer):
        '''
        Should check the user answer against the AI answer and return the difference between them
        '''
        difference = user_answer - ai_answer
        formatted_difference = str(difference)
        print(f"The difference between your guess and bot's answer is: {formatted_difference}") 
        


In [44]:
quiz_bot = HistoryQuiz()
question = quiz_bot.create_history_question(topic='Vietnam War')
question

---- History Question ------
Tokens Used: 56
	Prompt Tokens: 38
	Completion Tokens: 18
Successful Requests: 1
Total Cost (USD): $9.300000000000001e-05
----------


'What was the date when the United States officially withdrew its combat troops from the Vietnam War?'

In [45]:
bot_date_answer = quiz_bot.get_AI_answer(question)
bot_date_answer

Question=What was the date when the United States officially withdrew its combat troops from the Vietnam War?
Write a datetime string that matches the 
            following pattern: "%Y-%m-%dT%H:%M:%S.%fZ". Examples: 0557-05-31T08:21:02.140743Z, 0684-05-29T06:06:35.759850Z, 1280-02-19T18:55:10.996330Z
----------
Tokens Used: 156
	Prompt Tokens: 140
	Completion Tokens: 16
Successful Requests: 1
Total Cost (USD): $0.00024200000000000003
----------


datetime.datetime(1973, 3, 29, 0, 0)

In [46]:
user_date = quiz_bot.get_user_answer(question)

Question=What was the date when the United States officially withdrew its combat troops from the Vietnam War?


In [49]:
user_date

datetime.datetime(1973, 12, 31, 0, 0)

In [51]:
quiz_bot.check_user_answer(user_date, bot_date_answer)

The difference between your guess and bot's answer is: 277 days, 0:00:00
